In [9]:
import pandas as pd
import numpy as np
import warnings
import math
import scipy.stats
import statsmodels.api as sm
warnings.filterwarnings("ignore")

import os
os.chdir("/Users/Lenovo/Desktop/EBAC")

In [66]:
data = pd.read_csv("kc_house_data.csv")
data

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [108]:
Y = data['price'].values.reshape(-1,1)
features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'lat', 'long']
X = data[features].copy()
X = sm.add_constant(X)
X = X.values

In [110]:
XT_X = np.matmul(np.matrix.transpose(X), X)
XT_X_inv = np.linalg.inv(XT_X)
XT_Y = np.matmul(np.matrix.transpose(X), Y)
betas = np.matmul(XT_X_inv, XT_Y)

# Calculo de los pronosticos para Y de acuerdo a los coeficientes de regresion
Y_pred = np.matmul(X, betas)

# Calculo de residuales
Resid = Y - Y_pred

# Calculo de suma de residuales al cuadrado
RSS = float(np.matmul(np.matrix.transpose(Resid), Resid))

# Calculo de suma total de cuadrados
TSS = float(np.matmul(np.matrix.transpose(Y), Y) - len(Y)*(Y.mean()**2))

# Calculo de coeficiente de determinacion
R_cuad = float(1 - RSS/TSS)

# Calculo de coeficiente de determinacion ajustado
RSqAj = float(1 - (RSS / (X.shape[0] - X.shape[1])) / (TSS / X.shape[0] - 1))

# Calculo de varianza de error de regresion
s_cuad = RSS / (len(Y) - X.shape[1])

# Desviacion estandar del error de regresion
s = math.sqrt(s_cuad)

# Calculo de las t's estadisticas para cada coeficiente de regresion
result_t = []
for i in range(0, X.shape[1]):
    t = float(betas[i] / (s*math.sqrt(XT_X_inv[i][i])))
    result_t.append(t)
result_t

# Obtener valor critico de la t de Student de tablas
import scipy.stats

grados_libertad = len(Y) - X.shape[1]
# La t critica se obtendra a un nivel de confianza del 95% (Alfa = 5%)
t_critico = abs(scipy.stats.t.ppf(q = 0.025, df = grados_libertad))

## Criterio 1
for i in range(0, X.shape[1]):
    if abs(result_t[i] > t_critico):
        print("Beta", i, "es significativa") # Aqui se rechaza H0
    else:
        print("Beta", i, "NO es significativa") # Aqui NO se rechaza H0

## Criterio 2
# Calculo de valores p
for i in range(0, X.shape[1]):
    print("Valor p de Beta", i, ":", scipy.stats.t.sf(abs(result_t[i]), df = grados_libertad) * 2)

## Criterio 3
# Calculo de intervalos de confianza del 95% para el verdadero valor del coeficiente de cada Beta
for i in range(0, X.shape[1]):
    print("El valor de Beta", i, "se encuentra entre", float(betas[i]) - t_critico * s * math.sqrt(XT_X_inv[i][i]),
          "y", float(betas[i]) + t_critico * s * math.sqrt(XT_X_inv[i][i]))

Beta 0 NO es significativa
Beta 1 NO es significativa
Beta 2 NO es significativa
Beta 3 es significativa
Beta 4 NO es significativa
Beta 5 NO es significativa
Beta 6 es significativa
Beta 7 es significativa
Beta 8 es significativa
Beta 9 es significativa
Beta 10 es significativa
Beta 11 NO es significativa
Valor p de Beta 0 : 0.0
Valor p de Beta 1 : 1.5683673954192252e-53
Valor p de Beta 2 : 0.6212899800230343
Valor p de Beta 3 : 0.0
Valor p de Beta 4 : 0.6112292011242967
Valor p de Beta 5 : 5.883625993167733e-06
Valor p de Beta 6 : 1.3208697103460223e-232
Valor p de Beta 7 : 1.052456612158509e-156
Valor p de Beta 8 : 2.661064654662993e-106
Valor p de Beta 9 : 0.0
Valor p de Beta 10 : 0.0
Valor p de Beta 11 : 1.859913078707342e-90
El valor de Beta 0 se encuentra entre -60790720.642469816 y -55436267.03843839
El valor de Beta 1 se encuentra entre -34232.77028290146 y -26522.662267968422
El valor de Beta 2 se encuentra entre -4615.425207769238 y 7726.05370390948
El valor de Beta 3 se enc

In [112]:
# Comparacion de resultados contra reporte automatizado
import statsmodels.api as sm

regressor = sm.OLS(Y, X).fit()
print(regressor.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.673
Model:                            OLS   Adj. R-squared:                  0.672
Method:                 Least Squares   F-statistic:                     4034.
Date:                Sun, 07 Sep 2025   Prob (F-statistic):               0.00
Time:                        19:09:42   Log-Likelihood:            -2.9554e+05
No. Observations:               21613   AIC:                         5.911e+05
Df Residuals:                   21601   BIC:                         5.912e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -5.811e+07   1.37e+06    -42.547      0.0

In [114]:
# Mientras exista multicoordinalidad o dependencia entre variable regresoras no podemos obetener un resultado confiable
data = pd.DataFrame(X)
regresoras = data.iloc[:, 1:]
correlaciones = regresoras.corr()
print(correlaciones)

          1         2         3         4         5         6         7   \
1   1.000000  0.515884  0.576671  0.031703  0.175429 -0.006582  0.079532   
2   0.515884  1.000000  0.754665  0.087740  0.500653  0.063744  0.187737   
3   0.576671  0.754665  1.000000  0.172826  0.353949  0.103818  0.284611   
4   0.031703  0.087740  0.172826  1.000000 -0.005201  0.021604  0.074710   
5   0.175429  0.500653  0.353949 -0.005201  1.000000  0.023698  0.029444   
6  -0.006582  0.063744  0.103818  0.021604  0.023698  1.000000  0.401857   
7   0.079532  0.187737  0.284611  0.074710  0.029444  0.401857  1.000000   
8   0.028472 -0.124982 -0.058753 -0.008958 -0.263768  0.016653  0.045990   
9   0.356967  0.664983  0.762704  0.113621  0.458183  0.082775  0.251321   
10 -0.008931  0.024573  0.052529 -0.085683  0.049614 -0.014274  0.006157   
11  0.129473  0.223042  0.240223  0.229521  0.125419 -0.041910 -0.078400   

          8         9         10        11  
1   0.028472  0.356967 -0.008931  0.129473

In [116]:
X_Nueva = np.delete(X, 2, 1)
X_Nueva

array([[ 1.00000e+00,  3.00000e+00,  1.18000e+03, ...,  7.00000e+00,
         4.75112e+01, -1.22257e+02],
       [ 1.00000e+00,  3.00000e+00,  2.57000e+03, ...,  7.00000e+00,
         4.77210e+01, -1.22319e+02],
       [ 1.00000e+00,  2.00000e+00,  7.70000e+02, ...,  6.00000e+00,
         4.77379e+01, -1.22233e+02],
       ...,
       [ 1.00000e+00,  2.00000e+00,  1.02000e+03, ...,  7.00000e+00,
         4.75944e+01, -1.22299e+02],
       [ 1.00000e+00,  3.00000e+00,  1.60000e+03, ...,  8.00000e+00,
         4.75345e+01, -1.22069e+02],
       [ 1.00000e+00,  2.00000e+00,  1.02000e+03, ...,  7.00000e+00,
         4.75941e+01, -1.22299e+02]])

In [118]:
# Volvemos a correr el modelo
regressor = sm.OLS(Y, X_Nueva).fit()
print(regressor.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.673
Model:                            OLS   Adj. R-squared:                  0.672
Method:                 Least Squares   F-statistic:                     4437.
Date:                Sun, 07 Sep 2025   Prob (F-statistic):               0.00
Time:                        19:11:23   Log-Likelihood:            -2.9554e+05
No. Observations:               21613   AIC:                         5.911e+05
Df Residuals:                   21602   BIC:                         5.912e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -5.807e+07   1.36e+06    -42.607      0.0

# Conclusion
Para este ejercicio tome en cuenta las variables que use en la tarea anterior, en este caso vemos que nuestra R2 es de 67.3% por lo que no es muy buena, por lo que decidi quitar la variable "bathrooms" ya que tenia un P Value muy alto en comparacion a otra variables y tenia una correlacion del 75% con la variable 3. Al eliminarla de la ecuacion encontre que nuestra R2 no se afecto, se mantuvo en el mismo resultado.